In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/11/13 11:00:28 WARN Utils: Your hostname, hcode-ThinkPad-E14-Gen-3 resolves to a loopback address: 127.0.1.1; using 192.168.1.41 instead (on interface wlp3s0)
24/11/13 11:00:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/13 11:00:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/13 11:00:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
df = spark.read.csv("hw_200.csv", inferSchema=True, header=True)

In [11]:
# Read the CSV file into a PySpark DataFrame and display its schema.
df.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Height(Inches): double (nullable = true)
 |-- Weight(Pounds): string (nullable = true)



In [28]:
from pyspark.sql.functions import col, when, trim, udf
from pyspark.sql.types import DoubleType

In [19]:
df_ = df.withColumn(
    "interpretedHeight", when(trim(col("Height(Inches)")).rlike("\d+"), trim(col("Height(Inches)")).cast("double")).otherwise(None)
).withColumn(
    "interpretedWeight", when(trim(col("Weight(Pounds)")).rlike("\d+"), trim(col("Weight(Pounds)")).cast("double")).otherwise(None)
)

In [21]:
# Explore the data: Calculate basic statistics like average height and weight.
avg_height, avg_weight = df_.select(["interpretedHeight", "interpretedWeight"]).groupBy().avg().collect()[0] # column names 
print(f'''avg height : {avg_height}
avg_weight : {avg_weight}''')

avg height : 68.22174999999999
avg_weight : 127.39107142857151


In [23]:
# Filter the DataFrame to include only rows where the weight is above the average weight.
df_[df_["interpretedWeight"] > avg_weight].show()

+-----+--------------+--------------+-----------------+-----------------+
|Index|Height(Inches)|Weight(Pounds)|interpretedHeight|interpretedWeight|
+-----+--------------+--------------+-----------------+-----------------+
|    2|         71.52|        136.49|            71.52|           136.49|
|    3|          69.4|        153.03|             69.4|           153.03|
|    4|         68.22|        142.34|            68.22|           142.34|
|    5|         67.79|        144.30|            67.79|            144.3|
|    7|          69.8|        141.49|             69.8|           141.49|
|    8|         70.01|        136.46|            70.01|           136.46|
|   11|         66.49|        127.45|            66.49|           127.45|
|   16|         71.09|        140.00|            71.09|            140.0|
|   17|         66.46|        129.50|            66.46|            129.5|
|   18|         68.65|        142.97|            68.65|           142.97|
|   19|         71.23|        137.90| 

In [39]:
# bmiCalculate = udf(lambda weight, height: (weight / 2.205) / (height * 0.0254) ** 2, DoubleType())
bmiCalculate = udf(lambda weight, height: ((weight / 2.205) / (height * 0.0254) ** 2) if (weight and height) else None, DoubleType())

In [40]:
# Add a new column 'BMI' (Body Mass Index) using the formula: BMI = weight (kg) / (height (m))^2.
df_with_bmi = df_.withColumn("BMI", bmiCalculate(df_["interpretedWeight"], df_["interpretedHeight"]))
df_with_bmi.show()

+-----+--------------+--------------+-----------------+-----------------+------------------+
|Index|Height(Inches)|Weight(Pounds)|interpretedHeight|interpretedWeight|               BMI|
+-----+--------------+--------------+-----------------+-----------------+------------------+
|    1|         65.78|        112.99|            65.78|           112.99|18.355923679274014|
|    2|         71.52|        136.49|            71.52|           136.49|18.757277138145376|
|    3|          69.4|        153.03|             69.4|           153.03| 22.33477646774439|
|    4|         68.22|        142.34|            68.22|           142.34| 21.49945764296016|
|    5|         67.79|        144.30|            67.79|            144.3| 22.07288172769739|
|    6|          68.7|        123.30|             68.7|            123.3| 18.36426602913339|
|    7|          69.8|        141.49|             69.8|           141.49|20.414505961266148|
|    8|         70.01|        136.46|            70.01|           136.

In [37]:
# Save the transformed DataFrame as a Parquet file for efficient storage.
df_with_bmi.write.parquet("hw_200_parquet")

In [38]:
# Testing whether file saved correctly 
test_parquet = spark.read.parquet("hw_200_parquet")
test_parquet.show()

+-----+--------------+--------------+-----------------+-----------------+------------------+
|Index|Height(Inches)|Weight(Pounds)|interpretedHeight|interpretedWeight|               BMI|
+-----+--------------+--------------+-----------------+-----------------+------------------+
|    1|         65.78|        112.99|            65.78|           112.99|18.355923679274014|
|    2|         71.52|        136.49|            71.52|           136.49|18.757277138145376|
|    3|          69.4|        153.03|             69.4|           153.03| 22.33477646774439|
|    4|         68.22|        142.34|            68.22|           142.34| 21.49945764296016|
|    5|         67.79|        144.30|            67.79|            144.3| 22.07288172769739|
|    6|          68.7|        123.30|             68.7|            123.3| 18.36426602913339|
|    7|          69.8|        141.49|             69.8|           141.49|20.414505961266148|
|    8|         70.01|        136.46|            70.01|           136.